# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [19]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import pdist, squareform

In [20]:
data = pd.read_csv('/Users/Dinis/Ironhack/Labs/lab-intro-recommender-systems/data/customer_product_sales.csv')

In [21]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [52]:
customer_products = pd.DataFrame(data.groupby(['CustomerID','ProductName'])['Quantity'].sum().reset_index())
customer_products

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1
...,...,...,...
63623,98200,Vol Au Vents,50
63624,98200,Wasabi Powder,25
63625,98200,Wine - Fume Blanc Fetzer,25
63626,98200,Wine - Hardys Bankside Shiraz,25


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [53]:
# rows = the products, columns = customers, values = quantities of each product
# replace nulls with zeros = fillna method

product_by_customer = pd.pivot_table(customer_products, index =['CustomerID'], columns = ['ProductName'], 
                                     aggfunc=[np.sum], values = ['Quantity']).fillna(0)

product_by_customer

sum                                    \
                             Quantity                                     
ProductName Anchovy Paste - 56 G Tube Appetizer - Mini Egg Roll, Shrimp   
CustomerID                                                                
33                                0.0                               0.0   
200                               0.0                               0.0   
264                               0.0                               0.0   
356                               0.0                               0.0   
412                               0.0                               0.0   
...                               ...                               ...   
97928                             0.0                               0.0   
98069                             0.0                               0.0   
98159                             0.0                               0.0   
98185                             0.0                               0.0   
98200                             0.0                               0.0   

                                                                 \
                                                                  
ProductName Appetizer - Mushroom Tart Appetizer - Sausage Rolls   
CustomerID                                                        
33                                0.0                       0.0   
200                               0.0                       0.0   
264                               0.0                       0.0   
356                               0.0                       0.0   
412                               0.0                       0.0   
...                               ...                       ...   
97928                             0.0                      25.0   
98069                             0.0                      25.0   
98159                             0.0                       0.0   
98185                            25.0                      25.0   
98200                             0.0                       0.0   

                                                               \
                                                                
ProductName Apricots - Dried Apricots - Halves Apricots Fresh   
CustomerID                                                      
33                       1.0               0.0            0.0   
200                      0.0               0.0            0.0   
264                      0.0               1.0            1.0   
356                      0.0               0.0            0.0   
412                      1.0               0.0            0.0   
...                      ...               ...            ...   
97928                    0.0              50.0            0.0   
98069                    0.0              25.0            0.0   
98159                    0.0               0.0            0.0   
98185                    0.0              25.0            0.0   
98200                    0.0              25.0           25.0   

                                                                          ...  \
                                                                          ...   
ProductName Arizona - Green Tea Artichokes - Jerusalem Assorted Desserts  ...   
CustomerID                                                                ...   
33                          0.0                    0.0               1.0  ...   
200                         0.0                    0.0               0.0  ...   
264                         0.0                    0.0               0.0  ...   
356                         0.0                    0.0               0.0  ...   
412                         0.0                    0.0               0.0  ...   
...                         ...                    ...               ...  ...   
97928                      25.0                    0.0               0.0  ...   
98069                       0.0    

## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [55]:
# squareform = converts a vector-form distance vector to a sqare-form distance matrix, and vice-versa
# For our user similarity based recommender, we need to calculate the similarities between customers

customer_similarity_matrix = pd.DataFrame(1/(1 + squareform(pdist(product_by_customer, 'euclidean'))),
                            index=product_by_customer.index, columns=product_by_customer.index)
customer_similarity_matrix

cust_dist = pd.DataFrame(1/(1 + squareform(pdist(cust_prod_pivot, 'euclidean'))),
                         index=cust_prod_pivot.index, columns=cust_prod_pivot.index)

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.854082,0.859145,0.852783,0.856118,0.853239,0.849549,0.848801,0.851224,0.849892,...,0.855130,0.846812,0.850301,0.841254,0.852221,0.846485,0.852848,0.847637,0.856465,0.851614
200,0.854082,1.000000,0.857935,0.855900,0.856445,0.853129,0.863562,0.850521,0.857381,0.856621,...,0.864397,0.856014,0.862447,0.854629,0.856670,0.854046,0.868990,0.857312,0.861508,0.857184
264,0.859145,0.857935,1.000000,0.851801,0.860050,0.851262,0.855847,0.846677,0.854206,0.848997,...,0.861268,0.850766,0.854178,0.848507,0.858909,0.860153,0.859773,0.854246,0.864699,0.853668
356,0.852783,0.855900,0.851801,1.000000,0.852571,0.849431,0.853831,0.851902,0.847607,0.846328,...,0.857171,0.848015,0.852222,0.847706,0.851413,0.851289,0.851219,0.852260,0.860721,0.854377
412,0.856118,0.856445,0.860050,0.852571,1.000000,0.855755,0.858976,0.857221,0.854857,0.855991,...,0.859117,0.862847,0.854943,0.850044,0.858385,0.856629,0.855052,0.856608,0.865269,0.862253
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.846485,0.854046,0.860153,0.851289,0.856629,0.855545,0.856079,0.851250,0.853632,0.849471,...,0.857702,0.854124,0.850987,0.851150,0.862796,1.000000,0.862919,0.855458,0.858954,0.851527
98069,0.852848,0.868990,0.859773,0.851219,0.855052,0.852574,0.857118,0.851869,0.855343,0.855701,...,0.859570,0.855797,0.855409,0.847469,0.861720,0.862919,1.000000,0.859010,0.864271,0.858525
98159,0.847637,0.857312,0.854246,0.852260,0.856608,0.852782,0.854378,0.851396,0.855557,0.851331,...,0.859624,0.858006,0.853659,0.845236,0.855405,0.855458,0.859010,1.000000,0.861266,0.854102


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [56]:
# plugging in customer ID 33 and see what results we get for top5
similar_cust = list(customer_similarity_matrix[33].sort_values(ascending=False)[1:].head().index)

# similar_cust is a list containing the 5 customer IDs of the customers whose purchase behavior 
# is most similar to customer 33

print(f'5 customers whose purchase behavior is most similar to customer 33: {similar_cust}')

5 customers whose purchase behavior is most similar to customer 33: [60862, 27672, 6001, 79458, 33759]


## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [57]:
# Using the customer_products dataframe, we select just the purchases where the customer ID 
# is in our list of similar customers

sim_cust_prod = customer_products[customer_products['CustomerID'].isin(similar_cust)]
sim_cust_prod

,CustomerID,ProductName,Quantity
4418,6001,Apricots - Halves,2
4419,6001,Apricots Fresh,2
4420,6001,Arizona - Green Tea,2
4421,6001,Assorted Desserts,2
4422,6001,"Beef - Chuck, Boneless",2
...,...,...,...
50794,79458,"Wine - Cahors Ac 2000, Clos",21
50795,79458,Wine - Crozes Hermitage E.,21
50796,79458,Wine - Pinot Noir Latour,21
50797,79458,Wine - Redchard Merritt,42


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [58]:
# Aggregating on product name, summing up the total quantity purchased of each product by all similar customers
grouped = sim_cust_prod.groupby('ProductName').agg({'Quantity':'sum'})

# Ranking them by sorting in descending order by the total quantity
ranked_products = grouped.sort_values('Quantity', ascending=False).reset_index()
ranked_products.head()

,ProductName,Quantity
0,Wine - Redchard Merritt,59
1,Cassis,58
2,Bread - Calabrese Baguette,45
3,Wine - Crozes Hermitage E.,45
4,"Thyme - Lemon, Fresh",42


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [59]:
merged = pd.merge(ranked_products, pd.DataFrame(product_by_customer.T[33]), on='ProductName')
merged.columns = ['ProductName', 'Quantity', 'Purchased']
recs = merged[merged['Purchased']==0].head()

recs.head()

,ProductName,Quantity,Purchased
0,Wine - Redchard Merritt,59,0.0
2,Bread - Calabrese Baguette,45,0.0
4,"Thyme - Lemon, Fresh",42,0.0
9,Milk Powder,39,0.0
10,Ecolab - Lime - A - Way 4/4 L,39,0.0


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [78]:
# Creating an empty dictionary that will hold the recommendations for all customers.
recommendations = {}

# Creating a list of unique CustomerIDs to iterate over
customers = list(customer_products['CustomerID'].unique())

# Looping over the customer list for each and appending the results of each iteration to the dictionary 
for customer in customers:
    similar_cust = list(customer_similarity_matrix[customer].sort_values(ascending=False)[1:].head().index)
    sim_cust_prod = customer_products[customer_products['CustomerID'].isin(similar_cust)]
    grouped = pd.DataFrame(sim_cust_prod.groupby('ProductName').Quantity.sum())
    ranked_products = grouped.sort_values('Quantity', ascending=False).reset_index()
    
    merged = pd.merge(ranked_products, pd.DataFrame(product_by_customer.T[customer]), on='ProductName')
    merged.columns = ['ProductName', 'Quantity', 'Purchased']
    recs = merged[merged['Purchased']==0].head()
    customer_orders = merged[merged['Purchased']==0].head()
    
    user_recs = pd.DataFrame.from_dict(recommendations, orient='index').reset_index()
    user_recs.columns = ['CustomerID', 'Rec1', 'Rec2', 'Rec3', 'Rec4', 'Rec5']

user_recs.head()

####

    similar_cust = list(cust_dist[customer].sort_values(ascending=False)[1:].head().index)
    sim_cust_prod = customer_products[customer_products['CustomerID'].isin(similar_cust)]
    grouped = sim_cust_prod.groupby('ProductName').agg({'Quantity':'sum'})
    ranked_products = grouped.sort_values('Quantity', ascending=False).reset_index()
    
    merged = pd.merge(ranked_products, pd.DataFrame(cust_prod_pivot.T[customer]), on='ProductName')
    merged.columns = ['ProductName', 'Quantity', 'Purchased']
    recs = merged[merged['Purchased']==0].head()
    recommendations[customer] = list(recs['ProductName'])

user_recs = pd.DataFrame.from_dict(recommendations, orient='index').reset_index()
user_recs.columns = ['CustomerID', 'Rec1', 'Rec2', 'Rec3', 'Rec4', 'Rec5']
user_recs.head()

ValueError: Length mismatch: Expected axis has 1 elements, new values have 6 elements

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.